In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('schema_map.csv', sep='|', names=['form_version_id', 'path'])
df['element_name'] = df.path.str.split('.').apply(lambda x: x[-1])

df.head()

In [ ]:
df['has_name'] = df.element_name.str.lower().str.contains('name')
df['has_business'] = df.element_name.str.lower().str.contains('business')

names = df[(df.has_name) & (~df.has_business)]
# fvi is form_version_id
frequency_of_fvi = names.groupby('path')['element_name'].count()
frequency_of_fvi.sort_values(inplace=True, ascending=False)
frequency_of_fvi

In [ ]:
frequency_of_fvi.to_csv('paths.csv')